In [36]:
import pandas as pd
import bz2
import json
from os import listdir

In [37]:
from empath import Empath
lexicon = Empath()

In [66]:
# Generate a lexicon of keywords that relate to gun violence
lexicon.create_category("gun_violence", ["gun_violence","mass_shooting", "firearm", "firearm_shooting"], model="reddit")


["mass_shooting", "gun_violence", "mass_shootings", "gun_crime", "gun_ownership", "gun_crimes", "firearms", "firearm", "CCW_holders", "legal_gun_owners", "concealed_carriers", "violent_crime", "gun_owner", "accidental_shootings", "fire_arms", "gun_owners", "gun_death", "shootings", "gun_use", "police_shootings", "gun_control_laws", "home_invasions", "gun_deaths", "fire_arm", "US_police", "strict_gun_control", "active_shooters", "concealed_carry", "firearm_owners", "homicide", "home_invasion", "firearm_ownership", "mass_shooter", "violent_crimes", "accidental_shooting", "gun_ban", "handguns", "most_gun_owners", "school_shooting", "armed_criminals", "legal_guns", "responsible_gun_owner", "school_shootings", "assault_weapons", "illegal_guns", "mass_shooters", "law_enforcement_officers", "conceal_carry", "gun_possession", "legal_gun_ownership", "homicides", "related_crime", "strict_gun_laws", "CCW", "fully_automatic_weapons", "related_homicides", "gun_culture", "responsible_gun_ownership",

In [ ]:
def keyword(row):
    '''
    Filter the quotation of a particular row of a dataframe with a set of keywords.
    :param row: dataframe object
    :return: list of comma separated matching keywords
    '''
    strings = row['quotation']
    keywords = ["gun violence","mass shooting","firearm-related violence","firearm violence"]
    keyword = [key for key in keywords if key.upper() in strings.upper()]
    return ','.join(keyword)

In [ ]:
# Long one to execute (split dataset into pickles, identifies keywords present in each quote - want to modify,
# and saves to new file)... 
def process_chunk(chunk, i):
    print(f'Processing chunk with {len(chunk)} rows')
    
    # print(chunk.columns)
    # print(chunk.shape)
    # print(len(chunk[chunk.isna().any(axis=1)]))
    # print(chunk.sample(10))
    
    selected_columns = chunk[["speaker", "qids", "date", "quotation"]]
    filtered_chunk= selected_columns.copy()
    
    filtered_chunk['keyword'] = filtered_chunk.apply(keyword, axis=1)
    print(len(filtered_chunk[filtered_chunk.isna().any(axis=1)]))
    print(filtered_chunk.sample(5))
    print(filtered_chunk.shape)
    
    filtered_chunk.to_pickle("/Users/Justin/Desktop/processed_dfs/2019/" + str(i) + "chunk_with_keyword.pkl", compression='infer', protocol=4)

idx = 0
df_reader = pd.read_json('/Users/Justin/Desktop/Quote_Bank/quotes-2019.json-004.bz2', lines=True, compression='bz2', chunksize=1500000)
for chunk in df_reader:
    idx += 1
    process_chunk(chunk, idx)
    
    

In [35]:
# This is how to reopen a pickle, still need to write up filter functions ... 
small_df=pd.read_pickle('/Users/Justin/Desktop/processed_dfs/2019/1chunk_with_keyword.pkl', compression='infer')
small_df[small_df['keyword']!='']

,speaker,qids,date,quotation,keyword
2512,Eric Holder,"[Q312644, Q3591180]",2019-07-02 23:30:00,Crime Prevention and Research Center... has ex...,mass shooting
4416,Jeffrey Swanson,[Q22095876],2019-08-20 03:18:09,Here are some real cases of real people that m...,gun violence
5574,None,[],2019-01-10 00:35:56,I follow it every day so I knew we were tracki...,gun violence
8347,None,[],2019-04-11 00:00:00,This is so DISGUSTING. THIS is where you are p...,gun violence
9167,Alexandria Ocasio-Cortez,[Q55223040],2019-08-05 00:00:00,"Video games aren't causing mass shootings, whi...",mass shooting
...,...,...,...,...,...
1491742,None,[],2019-12-20 00:00:00,"As you have heard me state, our greatest chall...",gun violence
1493194,None,[],2019-06-07 06:50:30,doubt that UBI would significantly reduce the ...,mass shooting
1496991,Andrew Papachristos,[Q30557107],2019-03-07 20:07:12,There's a role for police in gun violence prev...,gun violence
1497601,None,[],2019-09-03 18:00:00,This decision reflects a clear reality -- Amer...,gun violence


In [50]:
for i in range(len(small_df)):
    lexicon.analyze("small_df.iloc[i]['quotation'].casefold()", categories=["gun_violence"])

In [70]:
lexicon.analyze(small_df.iloc[9167]['quotation'].casefold(), categories=["gun_violence"])

{'gun_violence': 0.0}

In [68]:
small_df.iloc[9167]['quotation'].casefold()

"video games aren't causing mass shootings, white supremacy is,"